In [1]:
from spade.behaviour import CyclicBehaviour
from spade.agent import Agent
from spade.message import Message

In [2]:
import time
import asyncio
import ast
import random

In [3]:
AMOUNT_AGENTS = 8
COUNTER = 0
NUMBER_OF_ITERATIONS = 20
SENDING_FLAGS = [0, 0, 0, 0, 0, 0, 0, 0]
SECRET = [0, 0, 0, 0, 0, 0, 0, 0]
TEMP_RESULT = [0., 0., 0., 0., 0., 0., 0., 0.]
NEIGHBORS = [0, 0, 0, 0, 0, 0, 0, 0]
RECEIVE_MESSAGE = [0, 0, 0, 0, 0, 0, 0, 0]

In [4]:
def foo(x):
    return x ** 2 + 2 * x + 1

In [5]:
class MyBehaviour(CyclicBehaviour):
    my_name = ""
    my_id = -1
    my_neighbors = []
    send_list = []

    def __init__(self, my_name, my_neighbors):
        self.my_neighbors = my_neighbors
        self.send_list = my_neighbors
        self.my_name = my_name
        self.my_id = int(self.my_name[10]) - 1
        global NEIGHBORS
        NEIGHBORS[self.my_id] = len(self.my_neighbors)

        super().__init__()

    def list_delete_items(self, whom):
        temp_queue = []
        for i in self.send_list:
            if i != whom:
                temp_queue.append(i)
        self.send_list = temp_queue

    def refresh_list_items(self):
        self.send_list = self.my_neighbors

    async def on_start(self):
        global SECRET
        SECRET[int(self.my_name[10]) - 1] = int(foo(random.uniform(0, 10)))
        print(str(self.my_name) + " " + str(SECRET[int(self.my_name[10]) - 1]))

        await asyncio.sleep(10)  # 10

    async def run(self):
        global SENDING_FLAGS, SECRET, COUNTER, TEMP_RESULT

        if SENDING_FLAGS[self.my_id] == 0:

            for neighbor in self.send_list:
                if random.random() < 0.95:
                    msg_s = Message(to=neighbor)

                    msg_s.body = str(SECRET[self.my_id] * random.uniform(0.9, 1.1))
                    await self.send(msg_s)
                    self.list_delete_items(neighbor)

            if len(self.send_list) == 0:
                SENDING_FLAGS[self.my_id] = 1
                self.refresh_list_items()

        if self.my_name == "user_name_1@404.city" and COUNTER == NUMBER_OF_ITERATIONS:
            result = SECRET[self.my_id]
            print('Answer = ' + str(result))

        message = await self.receive(timeout=1)
        if message:
            if self.my_name != message.sender:
                global RECEIVE_MESSAGE
                RECEIVE_MESSAGE[self.my_id] += 1

                TEMP_RESULT[self.my_id] += float(message.body) / (len(self.my_neighbors) + 1)


        global AMOUNT_AGENTS
        global NEIGHBORS
        if sum(SENDING_FLAGS) == AMOUNT_AGENTS and sum(RECEIVE_MESSAGE) == 14:
            SENDING_FLAGS = [0, 0, 0, 0, 0, 0, 0, 0]
            RECEIVE_MESSAGE = [0, 0, 0, 0, 0, 0, 0, 0]

            for i in range(AMOUNT_AGENTS):
                TEMP_RESULT[i] += (float(SECRET[i]) / (NEIGHBORS[i] + 1))

                SECRET[i] = TEMP_RESULT[i]
            TEMP_RESULT = [0, 0, 0, 0, 0, 0, 0, 0]
            COUNTER += 1

        await asyncio.sleep(1)  # 10

In [6]:
class DummyAgent(Agent):
    my_agent_name = ""
    neighbors = []

    async def setup(self):
        b = MyBehaviour(self.my_agent_name, self.neighbors)
        self.add_behaviour(b)

    def __init__(self, name, password, neighbors):
        super(DummyAgent, self).__init__(name, password)
        self.my_agent_name = name
        self.neighbors = neighbors

In [7]:
if __name__ == "__main__":
    dummy1 = DummyAgent("user_name_1@404.city", "my_password_1", ["user_name_2@404.city", "user_name_3@404.city",
                                                                 "user_name_4@404.city"])
    dummy2 = DummyAgent("user_name_2@404.city", "my_password_2", ["user_name_1@404.city"])
    dummy3 = DummyAgent("user_name_3@404.city", "my_password_3", ["user_name_1@404.city"])
    dummy4 = DummyAgent("user_name_4@404.city", "my_password_4", ["user_name_1@404.city", "user_name_5@404.city",
                                                                 "user_name_6@404.city", "user_name_7@404.city"])
    dummy5 = DummyAgent("user_name_5@404.city", "my_password_5", ["user_name_4@404.city"])
    dummy6 = DummyAgent("user_name_6@404.city", "my_password_6", ["user_name_4@404.city"])
    dummy7 = DummyAgent("user_name_7@404.city", "my_password_7", ["user_name_4@404.city", "user_name_8@404.city"])
    dummy8 = DummyAgent("user_name_8@404.city", "my_password_8", ["user_name_7@404.city"])

    dummy1.start()
    time.sleep(1)

    dummy2.start()
    time.sleep(1)

    dummy3.start()
    time.sleep(1)

    dummy4.start()
    time.sleep(1)

    dummy5.start()
    time.sleep(1)


    dummy6.start()
    time.sleep(1)

    dummy7.start()
    time.sleep(1)

    dummy8.start()

    while True:
        try:
            time.sleep(1)
        except KeyboardInterrupt:
            break

    dummy1.stop()
    dummy2.stop()
    dummy3.stop()
    dummy4.stop()
    dummy5.stop()
    dummy6.stop()
    dummy7.stop()
    dummy8.stop()

user_name_2@404.city 19
user_name_1@404.city 19
user_name_3@404.city 50
user_name_4@404.city 5
user_name_5@404.city 62
user_name_6@404.city 1
user_name_7@404.city 58
user_name_8@404.city 90
Answer = 30.492059034613852
Answer = 30.492059034613852
Answer = 30.492059034613852
Answer = 30.492059034613852
